# Topic Modeling Experiment with Lyrics from Billboard100 from 1965-2015
by Keith Truong

## Introduction

Topic modeling utilizes an algorithm to seek patterns of words. This method can be used to identify similar structures or topics among corpus, and to draw relationships among different documents (Micah Saxton, Topic Modeling Best Practices).

For this project, I want to apply this method to a dataset of songs charted on the Billboard100 from 1965 to 2015, retrived from Humanities Datasets in Context. The data set contains 5101 entries of song titles, as well as their artists' names, ranking on the Billboard100, year released, lyrics, and source. About 271 entries do not have the wanted lyrics information, because they are either left blank or catergorized as instrumental. 

This project attempts to illustrate the common themes and topics among popular songs in the US over 50 years. With the topic modeling approach, I hope to study the common song-writing word and topic. Besides studying the connections between artists and their songs, this project also wants to reflect the audience of these songs. The common threads among these musical expressions and preferences can offer a lens into understanding American pop culture from 1965 to 2015. 

A majority of the code in this project refers to the coding assignments from CLS160: Introduction to Digital Humanities taught by Dr. Micah Saxton in Fall 2023 at Tufts University.


## Code
### Set up

In [1]:
! pip install funcy
! pip install tzdata
! pip install --no-dependencies pyLDAvis
! pip install wget
! pip install gensim


In [2]:
from collections import defaultdict
import wget
from gensim import corpora, models
import pandas as pd
import pyLDAvis.gensim
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import spacy

In [3]:
# set up nlp pipline
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipes('ner', 'parser')

['ner', 'parser']

### Upload data

In [4]:
path = ".../"
file_name = 'billboard_lyrics_1964_2015.csv'
df = pd.read_csv('billboard_lyrics_1964_2015.csv', encoding = "ISO-8859-1")
df.head()

,Rank,Song,Artist,Year,Lyrics
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love you...
2,4,you were on my mind,we five,1965,when i woke up this morning you were on my min...
3,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss ...
4,6,downtown,petula clark,1965,when youre alone and life is making you lonely...


### Prepare data

In [5]:
df['Lyrics'] = df['Lyrics'].astype(str) 

In [6]:
print (df.dtypes)

Rank       int64
Song      object
Artist    object
Year       int64
Lyrics    object
dtype: object


In [7]:
# extract lemmas that are nouns, verbs, or adjectives
def process_text(text):
    """Remove new line characters and lemmatize text. Returns string of lemmas"""
    text = text.replace('\n', ' ')
    doc = nlp(text)
    tokens = [token for token in doc]
    no_stops = [token for token in tokens if not token.is_stop]
    no_punct = [token for token in no_stops if token.is_alpha]
    noun_verb_tokens = [token for token in no_punct if token.pos_ == 'NOUN' or token.pos_ == 'VERB']
    noun_verb_lemmas = [token.lemma_ for token in noun_verb_tokens]
    noun_verb_lemmas_lower = [lemma.lower() for lemma in noun_verb_lemmas]
    noun_verb_lemmas_string = ' '.join(noun_verb_lemmas_lower)
    return noun_verb_lemmas_string

In [8]:
# extract lemmas that are only nouns
def process_text_nouns(text):
    """Remove new line characters and lemmatize text. Returns string of lemmas"""
    text = text.replace('\n', ' ')
    doc = nlp(text)
    tokens = [token for token in doc]
    no_stops = [token for token in tokens if not token.is_stop]
    no_punct = [token for token in no_stops if token.is_alpha]
    noun_tokens = [token for token in no_punct if token.pos_ == 'NOUN']
    noun_lemmas = [token.lemma_ for token in noun_tokens]
    noun_lemmas_lower = [lemma.lower() for lemma in noun_lemmas]
    noun_lemmas_string = ' '.join(noun_lemmas_lower)
    return noun_lemmas_string

In [9]:
# extract lemmas that are only verbs
def process_text_verbs(text):
    """Remove new line characters and lemmatize text. Returns string of lemmas"""
    text = text.replace('\n', ' ')
    doc = nlp(text)
    tokens = [token for token in doc]
    no_stops = [token for token in tokens if not token.is_stop]
    no_punct = [token for token in no_stops if token.is_alpha]
    verb_tokens = [token for token in no_punct if token.pos_ == 'VERB']
    verb_lemmas = [token.lemma_ for token in verb_tokens]
    verb_lemmas_lower = [lemma.lower() for lemma in verb_lemmas]
    verb_lemmas_string = ' '.join(verb_lemmas_lower)
    return verb_lemmas_string

In [11]:
# extract lemmas
df['noun_verb_lemmas'] = df['Lyrics'].apply(process_text)
df['noun_verb_lemmas'].head()

0    sham domingo samudio do tre tell thing see hor...
1    sugar honey bunch know love help love elsein l...
2    wake morning mind mind get trouble whoaoh get ...
3    close eye kiss lip s tenderness fingertip try ...
4    life make downtown get worry noise hurry help ...
Name: noun_verb_lemmas, dtype: object

In [12]:
df['noun_lemmas'] = df['Lyrics'].apply(process_text_nouns)
df['noun_lemmas'].head()

0    sham domingo samudio tre thing horn jaw let ch...
1    sugar honey bunch elsein life picture finger e...
2    morning mind mind trouble worry wound corner p...
3    eye lip tenderness fingertip baby baby whoa fe...
4    life downtown worry noise hurry downtownjust m...
Name: noun_lemmas, dtype: object

In [13]:
df['verb_lemmas'] = df['Lyrics'].apply(process_text_verbs)
df['verb_lemmas'].head()

0    do tell see tell let come learn dance tell s p...
1    know love help love come leave kiss timeswhen ...
2    wake get whoaoh get whoaoh get bind go ease ea...
3    close kiss s try know lose lose go go go wohno...
4    make get help know listen lose forget forget w...
Name: verb_lemmas, dtype: object

In [14]:
# extract the data out of the DataFrame
nv_documents = df['noun_verb_lemmas'].to_list()
n_documents = df['noun_lemmas'].to_list()
v_documents = df['verb_lemmas'].to_list()

In [15]:
len(nv_documents[0])
len(n_documents[0])
len(v_documents[0])

75

In [18]:
# tokenize - the syntax below will create a list of lists
nv_texts =[
    [word for word in nv_document.lower().split()]
    for nv_document in nv_documents
]

n_texts =[
    [word for word in n_document.lower().split()]
    for n_document in n_documents
]

v_texts =[
    [word for word in v_document.lower().split()]
    for v_document in v_documents
]

In [20]:
# create a count of each token
frequency = defaultdict(int)
for nv_text in nv_texts:
  for token in nv_text:
    frequency[token] += 1
    
for n_text in n_texts:
  for token in n_text:
    frequency[token] += 1
    
for v_text in v_texts:
  for token in v_text:
    frequency[token] += 1

In [21]:
# remove words that appear only 1 time
nv_texts = [
    [token for token in text if frequency[token] > 1]
    for text in nv_texts
]

n_texts = [
    [token for token in text if frequency[token] > 1]
    for text in n_texts
]

v_texts = [
    [token for token in text if frequency[token] > 1]
    for text in v_texts
]

### Build topic model

In [22]:
# create a dictionary based off our texts
# The dictionary maps each token to a unique integer id
nv_dictionary = corpora.Dictionary(nv_texts)

In [23]:
# create a dictionary based off our texts
# The dictionary maps each token to a unique integer id
n_dictionary = corpora.Dictionary(n_texts)

In [24]:
# create a dictionary based off our texts
# The dictionary maps each token to a unique integer id
v_dictionary = corpora.Dictionary(v_texts)

In [39]:
# create a corpus based off our dictionary and our texts
nv_corpus = [nv_dictionary.doc2bow(nv_text) for nv_text in nv_texts]

In [44]:
# create a corpus based off our dictionary and our texts
n_corpus = [n_dictionary.doc2bow(n_text) for n_text in n_texts]

In [45]:
# create a corpus based off our dictionary and our texts
v_corpus = [v_dictionary.doc2bow(v_text) for v_text in v_texts]

In [40]:
# build LDA model
lda_model_nv = models.LdaModel(corpus=nv_corpus, id2word=nv_dictionary, num_topics=20, passes=70)

In [46]:
# build LDA model
lda_model_n = models.LdaModel(corpus=n_corpus, id2word=n_dictionary, num_topics=20, passes=70)

In [47]:
# build LDA model
lda_model_v = models.LdaModel(corpus=v_corpus, id2word=v_dictionary, num_topics=20, passes=70)

In [41]:
# explore topics
lda_model_nv.print_topics()

[(0,
  '0.218*"break" + 0.113*"heart" + 0.109*"help" + 0.032*"tear" + 0.031*"funk" + 0.019*"diva" + 0.016*"damage" + 0.013*"shout" + 0.010*"whoop" + 0.009*"aye"'),
 (1,
  '0.060*"fire" + 0.058*"burn" + 0.057*"lie" + 0.055*"stop" + 0.040*"stand" + 0.025*"move" + 0.021*"kick" + 0.015*"clap" + 0.013*"round" + 0.012*"make"'),
 (2,
  '0.087*"girl" + 0.063*"get" + 0.061*"know" + 0.052*"s" + 0.044*"ai" + 0.032*"tell" + 0.028*"man" + 0.022*"like" + 0.020*"want" + 0.020*"look"'),
 (3,
  '0.142*"go" + 0.089*"run" + 0.075*"know" + 0.068*"dance" + 0.054*"miss" + 0.051*"beat" + 0.024*"city" + 0.015*"night" + 0.014*"road" + 0.012*"trouble"'),
 (4,
  '0.138*"wait" + 0.031*"kid" + 0.029*"write" + 0.027*"power" + 0.023*"lip" + 0.020*"fight" + 0.019*"read" + 0.018*"care" + 0.017*"face" + 0.016*"sign"'),
 (5,
  '0.068*"boy" + 0.059*"hand" + 0.057*"turn" + 0.038*"hear" + 0.038*"play" + 0.035*"song" + 0.034*"music" + 0.033*"bring" + 0.028*"sing" + 0.018*"air"'),
 (6,
  '0.053*"know" + 0.051*"m" + 0.040*"ti

In [48]:
# explore topics
lda_model_n.print_topics()

[(0,
  '0.272*"way" + 0.154*"tonight" + 0.060*"friend" + 0.025*"day" + 0.014*"honey" + 0.014*"line" + 0.011*"time" + 0.011*"waitin" + 0.010*"alright" + 0.009*"cause"'),
 (1,
  '0.616*"baby" + 0.041*"woman" + 0.012*"lovin" + 0.010*"minute" + 0.010*"babe" + 0.009*"motion" + 0.008*"lover" + 0.007*"burn" + 0.006*"thing" + 0.005*"man"'),
 (2,
  '0.108*"hand" + 0.101*"let" + 0.078*"dance" + 0.058*"music" + 0.057*"floor" + 0.048*"party" + 0.022*"gon" + 0.018*"bass" + 0.015*"rhythm" + 0.015*"fun"'),
 (3,
  '0.072*"feelin" + 0.054*"tryin" + 0.033*"edge" + 0.031*"wing" + 0.030*"clap" + 0.029*"comin" + 0.027*"chorus" + 0.021*"highway" + 0.020*"goin" + 0.019*"hell"'),
 (4,
  '0.159*"boy" + 0.077*"day" + 0.075*"song" + 0.032*"babe" + 0.014*"rack" + 0.014*"jump" + 0.013*"number" + 0.013*"man" + 0.010*"hair" + 0.009*"weed"'),
 (5,
  '0.110*"heart" + 0.073*"world" + 0.043*"man" + 0.022*"time" + 0.021*"life" + 0.021*"people" + 0.017*"bit" + 0.015*"arm" + 0.014*"chance" + 0.012*"truth"'),
 (6,
  '0.119*

In [49]:
# explore topics
lda_model_v.print_topics()

[(0,
  '0.611*"love" + 0.049*"help" + 0.023*"hate" + 0.020*"blame" + 0.013*"get" + 0.009*"touch" + 0.009*"know" + 0.007*"care" + 0.006*"stand" + 0.005*"name"'),
 (1,
  '0.273*"feel" + 0.111*"fall" + 0.063*"believe" + 0.063*"try" + 0.041*"make" + 0.027*"dream" + 0.021*"wonder" + 0.020*"call" + 0.019*"m" + 0.019*"happen"'),
 (2,
  '0.278*"break" + 0.150*"kiss" + 0.080*"beat" + 0.059*"write" + 0.045*"fight" + 0.043*"build" + 0.041*"end" + 0.035*"mess" + 0.029*"learn" + 0.028*"matter"'),
 (3,
  '0.199*"find" + 0.125*"look" + 0.091*"fly" + 0.060*"see" + 0.047*"wake" + 0.047*"shine" + 0.025*"drink" + 0.022*"carry" + 0.022*"reach" + 0.016*"lose"'),
 (4,
  '0.187*"worry" + 0.164*"bout" + 0.073*"prove" + 0.072*"wash" + 0.047*"fool" + 0.039*"cross" + 0.035*"wind" + 0.021*"bury" + 0.019*"fire" + 0.016*"cast"'),
 (5,
  '0.727*"gon" + 0.024*"move" + 0.019*"lift" + 0.014*"wanna" + 0.011*"have" + 0.010*"celebrate" + 0.010*"meet" + 0.010*"tear" + 0.009*"m" + 0.009*"treat"'),
 (6,
  '0.369*"s" + 0.210*

In [42]:
# Find topics in each document
lda_model_nv.get_document_topics(nv_corpus[80])

[(2, 0.21636395),
 (6, 0.67062074),
 (9, 0.03466244),
 (15, 0.028915156),
 (16, 0.028671093),
 (18, 0.012890657)]

In [50]:
# Find topics in each document
lda_model_n.get_document_topics(n_corpus[80])

[(1, 0.028749341),
 (4, 0.510122),
 (6, 0.12708528),
 (7, 0.1561706),
 (15, 0.15908806)]

In [51]:
# Find topics in each document
lda_model_v.get_document_topics(v_corpus[80])

[(1, 0.17447914),
 (3, 0.1423452),
 (4, 0.021875653),
 (8, 0.032979436),
 (9, 0.15122499),
 (11, 0.07550787),
 (13, 0.020934427),
 (15, 0.16748421),
 (18, 0.048522145),
 (19, 0.15484223)]

### Visualize topic model with noun-and-verb corpus

In [52]:
# visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_nv, nv_corpus, nv_dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.241622 -0.091632       1        1  23.004974
2      0.233354  0.073899       2        1  13.492930
17     0.165105  0.165426       3        1   8.232604
12     0.211737 -0.075189       4        1   6.866312
11     0.106061 -0.263714       5        1   5.681864
8      0.096788  0.131135       6        1   5.376444
18     0.037767 -0.086571       7        1   4.664162
15     0.011687 -0.054078       8        1   4.394215
7      0.075436  0.077540       9        1   4.272179
14    -0.029971  0.238932      10        1   3.913778
5     -0.005861  0.117471      11        1   3.820581
13     0.062839 -0.123684      12        1   3.591037
19    -0.041779  0.039765      13        1   2.632583
1     -0.098344  0.026144      14        1   2.387703
3     -0.060373 -0.074371      15        1   2.241139
10    -0.174757 -0.063662      16        1   1.483284
4     -0.127624  0.029572      17        1   1.381019
16    -0.232935  0.042715      18        1   0.992378
0     -0.218108 -0.084503      19        1   0.987208
9     -0.252644 -0.025194      20        1   0.583607, topic_info=       Term          Freq         Total Category  logprob  loglift
49     love  16610.000000  16610.000000  Default  30.0000  30.0000
87     baby   8812.000000   8812.000000  Default  29.0000  29.0000
64     want   7348.000000   7348.000000  Default  28.0000  28.0000
50        m  16204.000000  16204.000000  Default  27.0000  27.0000
118     gon   5208.000000   5208.000000  Default  26.0000  26.0000
...     ...           ...           ...      ...      ...      ...
912   fruit     10.409400     18.839879  Topic20  -5.6387   4.5504
1920  hidin      9.283180     13.657293  Topic20  -5.7532   4.7576
5027    jar     10.448647     27.881592  Topic20  -5.6349   4.1622
234   child     14.574365    365.193688  Topic20  -5.3021   1.9225
108    whoa     10.935787    170.203435  Topic20  -5.5893   2.3987

[1336 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
175      19  0.967710   accord
7342      9  0.910722      ace
7263     14  0.985965    actin
1807      2  0.100612    adore
1807      4  0.693105    adore
...     ...       ...      ...
417       7  0.894949  youtake
3987     17  0.947558     yoyo
3371      1  0.054267     zone
3371      6  0.852768     zone
3371      9  0.093029     zone

[3045 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 3, 18, 13, 12, 9, 19, 16, 8, 15, 6, 14, 20, 2, 4, 11, 5, 17, 1, 10])

### Visualize topic model with noun-only corpus

In [53]:
# visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_n, n_corpus, n_dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
5     -0.222949  0.045179       1        1  9.695239
7     -0.051389 -0.281217       2        1  8.753782
15    -0.236753 -0.011171       3        1  8.184006
12     0.038109 -0.233314       4        1  7.480025
9     -0.179444  0.132432       5        1  7.293254
6     -0.102440 -0.037062       6        1  5.999554
1     -0.036886  0.041596       7        1  5.729138
10    -0.030056  0.120827       8        1  5.473325
16    -0.057711 -0.043391       9        1  5.465053
0     -0.146593 -0.003342      10        1  4.960838
18     0.030360  0.045327      11        1  4.927317
4     -0.011587 -0.077676      12        1  4.834010
2      0.106040 -0.054053      13        1  4.757360
14     0.144502 -0.048885      14        1  3.491158
17     0.058769  0.117531      15        1  2.771748
8      0.022937  0.145605      16        1  2.744488
19     0.189536  0.041118      17        1  2.171922
3      0.178180  0.059326      18        1  1.904669
13     0.146075  0.047640      19        1  1.786271
11     0.161297 -0.006470      20        1  1.576841, topic_info=        Term          Freq         Total Category  logprob  loglift
28      love  10203.000000  10203.000000  Default  30.0000  30.0000
48      baby   8961.000000   8961.000000  Default  29.0000  29.0000
23      girl   6316.000000   6316.000000  Default  28.0000  28.0000
70     night   4023.000000   4023.000000  Default  27.0000  27.0000
88       way   4690.000000   4690.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
166     shoe     26.632285    199.217314  Topic20  -5.0135   2.1375
713    mouth     25.910687    214.796949  Topic20  -5.0409   2.0347
2293  player     22.248454    109.223937  Topic20  -5.1933   2.5586
11     thing     29.421828   3281.627556  Topic20  -4.9138  -0.5646
339    brand     22.499316    166.138642  Topic20  -5.1821   2.1504

[1206 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
4388      2  0.936732    account
726      18  0.903814       ache
5666      4  0.991178      actin
7151     12  0.955266  addiction
3566     13  0.886354       afro
...     ...       ...        ...
3062     11  0.946918       yoyo
2586      4  0.154620       zone
2586     13  0.203447       zone
2586     18  0.634756       zone
7776      8  0.937580     zoomin

[2526 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 16, 13, 10, 7, 2, 11, 17, 1, 19, 5, 3, 15, 18, 9, 20, 4, 14, 12])

### Visualize topic model with verb-only corpus

In [54]:
# visualize
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_v, v_corpus, v_dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18    -0.229807 -0.088271       1        1  14.255369
9     -0.217964  0.055219       2        1  11.184256
11    -0.177483 -0.094844       3        1  10.127330
6     -0.153326  0.083735       4        1   6.133798
1     -0.107161  0.055676       5        1   5.853180
13    -0.076361 -0.080669       6        1   5.734860
12    -0.046576 -0.129794       7        1   5.255730
15    -0.090851  0.169412       8        1   5.050527
19    -0.111906 -0.021345       9        1   4.876883
7      0.005702  0.022554      10        1   4.607068
0      0.033498 -0.131536      11        1   4.294742
10    -0.037019 -0.001884      12        1   3.932016
3     -0.004668  0.247169      13        1   3.929643
17     0.055548 -0.054073      14        1   3.488153
5      0.123219 -0.212004      15        1   2.686090
16     0.176518  0.135593      16        1   2.385685
8      0.174096  0.089380      17        1   2.266843
2      0.201098 -0.025625      18        1   1.800334
14     0.221355  0.010660      19        1   1.453158
4      0.262088 -0.029356      20        1   0.684336, topic_info=        Term          Freq         Total Category  logprob  loglift
23         m  14745.000000  14745.000000  Default  30.0000  30.0000
22      love   6486.000000   6486.000000  Default  29.0000  29.0000
30      want   7393.000000   7393.000000  Default  28.0000  28.0000
0       come   6711.000000   6711.000000  Default  27.0000  27.0000
59       gon   4830.000000   4830.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
1280  accuse      2.565319     10.631210  Topic20  -6.4918   3.5628
283    steal      5.768475    163.161516  Topic20  -5.6814   1.6421
4      learn      8.065525    394.291367  Topic20  -5.3462   1.0950
561      eat      3.748420    213.788535  Topic20  -6.1125   0.9408
1529   check      2.954178    208.384720  Topic20  -6.3506   0.7283

[1185 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
993      18  0.438136    abound
1279     17  0.092573     abuse
1279     20  0.833157     abuse
1280      4  0.658439    accuse
1280     20  0.282188    accuse
...     ...       ...       ...
1211      9  0.847505        yo
2518     10  0.577857  youcause
2518     15  0.397277  youcause
2875      8  0.897105   youjust
1591      9  0.929502   youwell

[2130 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 10, 12, 7, 2, 14, 13, 16, 20, 8, 1, 11, 4, 18, 6, 17, 9, 3, 15, 5])

## Discussion

### Data set

The raw data set was manually modified before uploading to this program. Out of 5101, 271 entries were removed because they did not have the wanted lyrics information, either because they were left blank or categorized as instrumental. Also of these 271 removed entries, less than 10 entries are songs with lyrics in non-Latin alphabets. There were attempts to upload these entries but the Topic Modeling algorithm does not seem to work effectively with transcribed-not translated-lyrics. After some research, I identified the issues: the current encoding has issues reading these entries. I tried different encoding options but it was unsuccessful. For future direction, I would like to solve this issue. 

### Corpus

After the lyrics data was uploaded to notebook and topic modeling was conducted for testing, the corpus was strongly impacted by the extraneous data, such as prepositions and conjunctions. Therefore, while preparing the data, I also removed all tokens that are not a noun or a verb. I wanted to look at adjectives as well but the spaCy NLP does not include adjectives in its part-of-speech labels. I also created two additional corpora that are noun-only and verb-only to observe how Gensim handles different part-of-speech corpora. 

### Analysis of Topic Modeling Artifacts

#### Noun-and-verb corpus

The topic modeling of noun-and-verb corpus results in a diverse set of topics. The topics on top of the PC1 axis range with a smooth transition among topics of money, partying, violence, sex, male/female identifier/reference, and body parts. While the topics below the PC1 axis focus more on emotions (sadness as a majority), nature, cityscapes, breaking up, and making love. 


#### Noun-only corpus

The topic modeling of noun-only corpus results in a less coherent set of topics. Out of 20 topics generated, 18 topics are located on top of the PC1 axis, with mostly mental and physical associations of love, nature, and life. The two topics located below the PC1 axis are more aggressive in their grouping of salient terms.

#### Verb-only corpus

The topic modeling of verb-only corpus results in the least coherent set of topics. Most of the topics cluster on the right side of the PC2 axis. Surprisingly, not a lot of these topics show evident themes of violence, sex, or love like the previous experiments. 

### Future Direction

I would like to look at this dataset based on the year of release and observe the changes in topics over time. A strategy for that project might start with creating different topic modeling for songs charted each year, then figure out a way to define each topic effectively to rank their frequency. 